# basic gnn
---

In [1]:
# Load the alexandria, model
import mylib.datasets.alexandria as alex
import mylib.models.basic_gnn as gnn

# Load the dataset
dataset = alex.Dataset(root="/home/gengyao/matbench", partial=(0.0,0.05), tensorize_work=True)
# Initialize the model
model = gnn.Model(device='cuda')
print(dataset, model)

/home/gengyao/matbench/.pixi/envs/default/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<mylib.datasets.alexandria.Dataset object at 0x1553f7553590> Model(
  (conv1): GCNConv(4, 12)
  (conv2): GCNConv(12, 24)
  (conv3): GCNConv(24, 48)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=48, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=2, bias=True)
  (lk): LeakyReLU(negative_slope=0.1)
)


## model training

In [ ]:
import torch as tc
import torch.nn as nn
from torch.optim import SGD

optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
loss_fn = nn.L1Loss()
batch_size = 4
batch_size_cnt = 0
loss = 0
# training loop
for epoch in range(100):
    for sample in dataset:
        y_bar = tc.tensor([[sample[0], sample[1]]]).to(model.device)
        y = model.forward(sample[3], sample[4], sample[5])
        loss += loss_fn(y, y_bar)
        batch_size_cnt += 1
        if batch_size_cnt == batch_size:
            # backpropagation
            print(f"epoch:{epoch}, loss:{loss}")
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss = 0
            batch_size_cnt = 0

In [59]:
# try torch_geometric dataloader.
from torch_geometric.loader import DataLoader
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
for batch in dataloader:
    print(batch)
    break

[tensor([-19.0483]), tensor([1.5895]), tensor([[[0.0000, 4.7247, 4.7247],
         [4.7247, 0.0000, 4.7247],
         [4.7247, 4.7247, 0.0000]]]), tensor([[[38.0000,  3.5356,  5.9137,  5.9137],
         [38.0000,  5.9137,  5.9137,  3.5356],
         [38.0000,  5.9137,  3.5356,  5.9137],
         [38.0000,  3.5356,  3.5356,  3.5356],
         [25.0000,  7.0870,  7.0870,  7.0870],
         [75.0000,  0.0000,  0.0000,  0.0000]]]), tensor([[[0, 1, 2, 3, 4, 4, 5, 5],
         [0, 1, 2, 3, 4, 5, 4, 5]]]), tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.4239, 1.4239, 0.0000]])]
